In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict
import matplotlib.pyplot as plt

import matplotlib as mpl
from implicit import bpr
import seaborn as sns
import pandas as pd
import numpy as np
HEAD = ["submissionDate","institution","major","degree","notif_result","notif_date","gpa","gre","gre_w"]
data = pd.read_csv('../data/cleaned_data/cleaned_grad_cafe_admissions.csv',)

c:\Users\solit\anaconda3\envs\LSTM_CNN\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data.tail(6)

,Unnamed: 0,submissionDate,institution,major,degree,notif_result,notif_date,gpa,gre,gre_w
152402,152402,2020-04-01,UC San Diego (UCSD),Electrical And Computer Engineering,Masters,Accepted,2020-04-01,8.12,325.0,4.0
152403,152403,2020-04-01,Cornell University,Horticulture,PhD,Rejected,2020-03-31,3.95,321.0,4.5
152404,152404,2020-04-01,University Of Western Ontario,Speech Language Pathology,Masters,Wait listed,2020-04-01,3.95,306.0,5.0
152405,152405,2020-04-01,UC San Diego (UCSD),Electrical And Computer Engineering,Masters,Accepted,2020-04-01,3.67,320.0,3.5
152406,152406,2020-04-02,UMass-Amherst,Computer Science,Masters,Accepted,2020-04-02,3.60,320.0,3.0
152407,152407,2020-04-02,University Of Pennsylvania,Chemical And Biomolecular Engineering,Masters,Other,2020-04-02,3.48,309.0,3.0


### Filter the data 

In [13]:
# replace ['E-mail','Wait listed','Other'] with "Rejected"
datafiltered=data.replace(to_replace=['E-mail','Wait listed','Other'],value='Rejected')
# filter data with unnormal results
datafiltered=datafiltered.loc[data['notif_result'].isin(['Accepted','Rejected'])] 
# filter data with unnormal gpa
datafiltered=datafiltered[datafiltered["gpa"].map(lambda x:x>0 and x<4.0)]
# filter data with unnormal gre
datafiltered=datafiltered[datafiltered["gre"].map(lambda x:x>300 and x<=340)]
datafiltered=datafiltered[datafiltered["gre_w"].map(lambda x:x>0.0 and x<=5.0)]
# data length
print("data only contains accepted and rejected:{}".format(len(datafiltered)))
# Split the data into training and testing sets
X = datafiltered.iloc[:,[1,2,3,4,6,7,8,9]]  # select all columns except the last one
y = datafiltered.iloc[:, 5]   # select the last column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

data only contains accepted and rejected:96314


### University Rank

In [51]:
university_rank=pd.read_csv('../data/university_rank/university_rank.csv',names=['rank','Uni_Name','Country',\
                            '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
university_rank.head()

,rank,Uni_Name,Country,0,1,2,3,4,5,6,7,8,9,10
0,1,Harvard University,USA,1,1,1,1,1,1,1,1,3,100.00,2015
1,2,Stanford University,USA,2,9,2,4,5,3,3,4,10,98.66,2015
2,3,Massachusetts Institute of Technology,USA,3,3,11,2,15,2,2,2,1,97.54,2015
3,4,University of Cambridge,United Kingdom,1,2,10,5,11,6,12,13,48,96.81,2015
4,5,University of Oxford,United Kingdom,2,7,13,10,7,12,7,9,15,96.46,2015


In [52]:
dict_university_rank=defaultdict(list)
university_set=set()
university_name=[]
university_value=[]
for i in range(len(university_rank)):
    uni_name=str(university_rank.iloc[i,1])
    if i<50:
        uni_rank=int(university_rank.iloc[i,0])//10
    elif i<100:
        uni_rank=5
    elif i<200:
        uni_rank=6
    elif i<500:
        uni_rank=7
    else:
        uni_rank=8
    dict_university_rank[uni_name]=uni_rank
    if uni_name not in university_set:
        university_set.add(uni_name)
        university_name.append(uni_name)
        university_value.append(uni_rank)


In [55]:
data_UniRank = datafiltered[datafiltered["institution"].map(lambda x: x in university_name)]
data_UniRank = data_UniRank.replace(to_replace=university_name,value=university_value)
print("Length of data which are interacted with university rank is:{}".format(len(data_UniRank)))
data_UniRank.tail(5)


Length of data which are interacted with university rank is:16777


,Unnamed: 0,submissionDate,institution,major,degree,notif_result,notif_date,gpa,gre,gre_w
152367,152367,2020-04-01,7,Film Studies,Masters,Rejected,2020-02-11,2.94,302.0,4.0
152370,152370,2020-04-01,6,Clinical Psychology,PhD,Rejected,2020-01-22,3.87,306.0,4.0
152381,152381,2020-04-01,6,Mathematics,PhD,Accepted,2020-04-01,3.63,320.0,5.0
152392,152392,2020-04-01,5,Computer Science,Masters,Accepted,2020-03-31,3.10,315.0,4.5
152403,152403,2020-04-01,1,Horticulture,PhD,Rejected,2020-03-31,3.95,321.0,4.5
